# VCF Benchmarking Statistics Analysis

This notebook performs comprehensive VCF statistics on CASTLE benchmarking truth sets for DeepSomatic model training and evaluation.

## Analysis Overview
- **Whole-Landscape Analysis**: Cross-sample comparisons and aggregate statistics
- **Per-Sample Analysis**: Individual sample characterization
- **Statistics Included**:
  - Variant type distribution (SNP/INDEL counts and percentages)
  - Chromosome distribution (natural sort order, counts and percentages)
  - FILTER label statistics
  - Ti/Tv ratio (transition/transversion)
  - Base change spectrum (6-type mutation categories)
  - Indel size distribution

## 1. Setup

Import libraries and configure paths.

In [ ]:
import sys
import re
from pathlib import Path
from collections import defaultdict
from typing import Dict, List, Tuple, Optional, Any

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from cyvcf2 import VCF

# Paths
BENCHMARKING_BASE = Path('/t9k/mnt/WorkSpace/data/ngs/xuzhenyu/CASTLE/benchmarking')
OUTPUT_DIR = Path('/t9k/mnt/hdd/work/Vax/pipeline/rnadnavar/notebook/castle_statistics_output')
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print('✓ Setup complete')

## 2. Configuration

Define benchmarking categories, samples, and VCF file paths.

In [ ]:
# Benchmarking categories
BENCHMARK_CATEGORIES = [
    'DeepSomatic_multicancer-model_benchmark',
    'DeepSomatic_HCC1395-model_benchmark'
]

# Expected samples
SAMPLES = ['H1437', 'H2009', 'HCC1395', 'HCC1937', 'HCC1954', 'HG008', 'Hs578']

# VCF file paths
VCF_PATHS = [
    # HCC1395-model benchmark
    BENCHMARKING_BASE / 'DeepSomatic_HCC1395-model_benchmark/vcfs/H1437_DeepSomatic_HCC1395-model.vcf.gz',
    BENCHMARKING_BASE / 'DeepSomatic_HCC1395-model_benchmark/vcfs/H2009_DeepSomatic_HCC1395-model.vcf.gz',
    BENCHMARKING_BASE / 'DeepSomatic_HCC1395-model_benchmark/vcfs/HCC1395_DeepSomatic_HCC1395-model.vcf.gz',
    BENCHMARKING_BASE / 'DeepSomatic_HCC1395-model_benchmark/vcfs/HCC1937_DeepSomatic_HCC1395-model.vcf.gz',
    BENCHMARKING_BASE / 'DeepSomatic_HCC1395-model_benchmark/vcfs/HCC1954_DeepSomatic_HCC1395-model.vcf.gz',
    # multicancer-model benchmark
    BENCHMARKING_BASE / 'DeepSomatic_multicancer-model_benchmark/vcfs/H1437_DeepSomatic_multicancer-model.vcf.gz',
    BENCHMARKING_BASE / 'DeepSomatic_multicancer-model_benchmark/vcfs/H2009_DeepSomatic_multicancer-model.vcf.gz',
    BENCHMARKING_BASE / 'DeepSomatic_multicancer-model_benchmark/vcfs/HCC1395_DeepSomatic_multicancer-model.vcf.gz',
    BENCHMARKING_BASE / 'DeepSomatic_multicancer-model_benchmark/vcfs/HCC1937_DeepSomatic_multicancer-model.vcf.gz',
    BENCHMARKING_BASE / 'DeepSomatic_multicancer-model_benchmark/vcfs/HCC1954_DeepSomatic_multicancer-model.vcf.gz',
]

# Validate paths
valid_vcfs = [p for p in VCF_PATHS if p.exists()]
missing_vcfs = [p for p in VCF_PATHS if not p.exists()]

print(f'✓ Found {len(valid_vcfs)} VCF files')
if missing_vcfs:
    print(f'⚠️  Missing {len(missing_vcfs)} files:')
    for p in missing_vcfs[:5]:
        print(f'   - {p.name}')

## 3. Helper Functions

### 3.1 Chromosome Natural Sort Utility

In [ ]:
def natural_sort_chromosomes(chromosomes: List[str]) -> List[str]:
    """
    Sort chromosomes in natural order: 1, 2, ..., 22, X, Y, M/MT.
    Handles both 'chr' prefixed and non-prefixed formats.
    """
    def chrom_sort_key(chrom: str) -> Tuple:
        # Remove 'chr' prefix if present
        c = chrom.lower().replace('chr', '')
        
        # Define order for special chromosomes
        special_order = {'x': 23, 'y': 24, 'm': 25, 'mt': 25}
        
        if c in special_order:
            return (special_order[c], 0, chrom)
        
        # Try to parse as integer
        try:
            num = int(c)
            return (num, 0, chrom)
        except ValueError:
            # Unknown chromosome, sort after known ones
            return (100, 0, chrom)
    
    return sorted(chromosomes, key=chrom_sort_key)


def get_chromosome_order() -> List[str]:
    """Return standard chromosome order for consistent plotting."""
    return [f'chr{i}' for i in range(1, 23)] + ['chrX', 'chrY', 'chrM']


# Test natural sort
test_chroms = ['chr2', 'chr10', 'chrX', 'chr1', 'chrY', 'chr22', 'chrM']
print(f'Natural sort test: {natural_sort_chromosomes(test_chroms)}')

In [ ]:
def inspect_vcf_header(vcf_path: Path) -> Dict[str, Any]:
    """
    Inspect VCF header to discover available fields.
    
    Returns:
        Dict with 'info_fields', 'format_fields', 'filter_fields', 'samples'
    """
    vcf = VCF(str(vcf_path))
    samples = list(vcf.samples)  # Get samples before iterating
    
    info_fields = {}
    format_fields = {}
    filter_fields = {}
    
    for field in vcf.header_iter():
        header_type = field['HeaderType']
        
        if header_type == 'INFO':
            try:
                info_fields[field['ID']] = {
                    'type': field['Type'],
                    'description': field['Description']
                }
            except (KeyError, TypeError):
                info_fields[field['ID']] = {'type': 'Unknown', 'description': ''}
                
        elif header_type == 'FORMAT':
            try:
                format_fields[field['ID']] = {
                    'type': field['Type'],
                    'description': field['Description']
                }
            except (KeyError, TypeError):
                format_fields[field['ID']] = {'type': 'Unknown', 'description': ''}
                
        elif header_type == 'FILTER':
            try:
                filter_fields[field['ID']] = field['Description']
            except (KeyError, TypeError):
                filter_fields[field['ID']] = ''
    
    vcf.close()
    
    return {
        'info_fields': info_fields,
        'format_fields': format_fields,
        'filter_fields': filter_fields,
        'samples': samples,
        'path': vcf_path
    }


def summarize_vcf_fields(vcf_paths: List[Path]) -> pd.DataFrame:
    """Summarize available fields across all VCF files."""
    summaries = []
    
    for vcf_path in vcf_paths:
        if not vcf_path.exists():
            continue
        
        header = inspect_vcf_header(vcf_path)
        summaries.append({
            'File': vcf_path.name,
            'INFO_Fields': len(header['info_fields']),
            'FORMAT_Fields': len(header['format_fields']),
            'FILTER_Fields': len(header['filter_fields']),
            'Samples': len(header['samples']),
            'Has_GT': 'GT' in header['format_fields'],
            'Has_DP': 'DP' in header['format_fields'],
            'Has_AF': 'AF' in header['format_fields'],
            'Has_AD': 'AD' in header['format_fields'],
        })
    
    return pd.DataFrame(summaries)


# Inspect first VCF to understand structure
if valid_vcfs:
    sample_header = inspect_vcf_header(valid_vcfs[0])
    print(f"Sample VCF: {valid_vcfs[0].name}")
    print(f"  INFO fields: {list(sample_header['info_fields'].keys())[:10]}...")
    print(f"  FORMAT fields: {list(sample_header['format_fields'].keys())}")
    print(f"  FILTER fields: {list(sample_header['filter_fields'].keys())}")
    print(f"  Samples: {sample_header['samples']}")

### 3.3 Core Statistics Functions

In [ ]:
class VCFBenchmarkStats:
    """
    Extract comprehensive statistics from a VCF file.
    Designed to work with any VCF format (universal, no pipeline-specific dependencies).
    """
    
    # Base change categories (complement-collapsed for standard mutation spectrum)
    BASE_CHANGE_CATEGORIES = {
        ('C', 'A'): 'C>A', ('G', 'T'): 'C>A',
        ('C', 'G'): 'C>G', ('G', 'C'): 'C>G',
        ('C', 'T'): 'C>T', ('G', 'A'): 'C>T',
        ('T', 'A'): 'T>A', ('A', 'T'): 'T>A',
        ('T', 'C'): 'T>C', ('A', 'G'): 'T>C',
        ('T', 'G'): 'T>G', ('A', 'C'): 'T>G',
    }
    
    TRANSITIONS = {('A', 'G'), ('G', 'A'), ('C', 'T'), ('T', 'C')}
    TRANSVERSIONS = {('A', 'C'), ('C', 'A'), ('A', 'T'), ('T', 'A'),
                     ('G', 'C'), ('C', 'G'), ('G', 'T'), ('T', 'G')}
    
    # Indel size bins
    INDEL_SIZE_BINS = ['1', '2', '3', '4-14', '15-29', '30-49', '>50']
    
    def __init__(self, vcf_path: Path):
        self.vcf_path = vcf_path
        self.stats = None
    
    @staticmethod
    def categorize_indel_size(size: int) -> str:
        """
        Categorize indel size into bins.
        
        Args:
            size: Indel size (positive for insertion, negative for deletion)
        
        Returns:
            Bin label: '1', '2', '3', '4-14', '15-29', '30-49', '>50'
        """
        abs_size = abs(size)
        
        if abs_size == 1:
            return '1'
        elif abs_size == 2:
            return '2'
        elif abs_size == 3:
            return '3'
        elif 4 <= abs_size <= 14:
            return '4-14'
        elif 15 <= abs_size <= 29:
            return '15-29'
        elif 30 <= abs_size <= 49:
            return '30-49'
        else:  # >= 50
            return '>50'
        
    def extract_all_stats(self) -> Dict[str, Any]:
        """Extract all statistics from the VCF file."""
        vcf = VCF(str(self.vcf_path))
        
        stats = {
            'total': 0,
            'snps': 0,
            'indels': 0,
            'other': 0,
            'chromosomes': defaultdict(int),
            'filters': defaultdict(int),
            'transitions': 0,
            'transversions': 0,
            'base_changes': defaultdict(int),
            'indel_sizes': [],
            'qual_scores': [],
        }
        
        for variant in vcf:
            stats['total'] += 1
            
            # Chromosome
            chrom = variant.CHROM
            stats['chromosomes'][chrom] += 1
            
            # FILTER
            filt = variant.FILTER if variant.FILTER else 'PASS'
            stats['filters'][filt] += 1
            
            # QUAL score
            if variant.QUAL is not None and variant.QUAL != -1:
                stats['qual_scores'].append(variant.QUAL)
            
            # Variant type
            ref = variant.REF.upper()
            alt = variant.ALT[0].upper() if variant.ALT else ''
            
            if variant.is_snp:
                stats['snps'] += 1
                
                # Ti/Tv calculation
                if (ref, alt) in self.TRANSITIONS:
                    stats['transitions'] += 1
                elif (ref, alt) in self.TRANSVERSIONS:
                    stats['transversions'] += 1
                
                # Base change spectrum
                if (ref, alt) in self.BASE_CHANGE_CATEGORIES:
                    category = self.BASE_CHANGE_CATEGORIES[(ref, alt)]
                    stats['base_changes'][category] += 1
                    
            elif variant.is_indel:
                stats['indels'] += 1
                # Indel size: positive = insertion, negative = deletion
                indel_size = len(alt) - len(ref)
                stats['indel_sizes'].append(indel_size)
            else:
                stats['other'] += 1
        
        vcf.close()
        
        # Convert defaultdicts to regular dicts
        stats['chromosomes'] = dict(stats['chromosomes'])
        stats['filters'] = dict(stats['filters'])
        stats['base_changes'] = dict(stats['base_changes'])
        
        # Calculate Ti/Tv ratio
        if stats['transversions'] > 0:
            stats['titv_ratio'] = stats['transitions'] / stats['transversions']
        else:
            stats['titv_ratio'] = None
        
        self.stats = stats
        return stats
    
    def get_variant_type_summary(self) -> pd.DataFrame:
        """Get variant type counts and percentages."""
        if not self.stats:
            self.extract_all_stats()
        
        total = self.stats['total']
        data = {
            'Type': ['SNP', 'INDEL', 'Other', 'Total'],
            'Count': [
                self.stats['snps'],
                self.stats['indels'],
                self.stats['other'],
                total
            ],
            'Percentage': [
                self.stats['snps'] / total * 100 if total > 0 else 0,
                self.stats['indels'] / total * 100 if total > 0 else 0,
                self.stats['other'] / total * 100 if total > 0 else 0,
                100.0
            ]
        }
        df = pd.DataFrame(data)
        df['Percentage'] = df['Percentage'].round(2)
        return df
    
    def get_chromosome_summary(self) -> pd.DataFrame:
        """Get chromosome distribution with counts and percentages (natural sort)."""
        if not self.stats:
            self.extract_all_stats()
        
        total = self.stats['total']
        chroms = natural_sort_chromosomes(list(self.stats['chromosomes'].keys()))
        
        data = {
            'Chromosome': chroms,
            'Count': [self.stats['chromosomes'][c] for c in chroms],
            'Percentage': [self.stats['chromosomes'][c] / total * 100 if total > 0 else 0 for c in chroms]
        }
        df = pd.DataFrame(data)
        df['Percentage'] = df['Percentage'].round(2)
        return df
    
    def get_filter_summary(self) -> pd.DataFrame:
        """Get FILTER label distribution with counts and percentages."""
        if not self.stats:
            self.extract_all_stats()
        
        total = self.stats['total']
        filters = sorted(self.stats['filters'].keys())
        
        data = {
            'Filter': filters,
            'Count': [self.stats['filters'][f] for f in filters],
            'Percentage': [self.stats['filters'][f] / total * 100 if total > 0 else 0 for f in filters]
        }
        df = pd.DataFrame(data)
        df['Percentage'] = df['Percentage'].round(2)
        return df.sort_values('Count', ascending=False).reset_index(drop=True)
    
    def get_base_change_summary(self) -> pd.DataFrame:
        """Get 6-type base change spectrum with counts and percentages."""
        if not self.stats:
            self.extract_all_stats()
        
        snp_total = self.stats['snps']
        categories = ['C>A', 'C>G', 'C>T', 'T>A', 'T>C', 'T>G']
        
        data = {
            'Base_Change': categories,
            'Count': [self.stats['base_changes'].get(c, 0) for c in categories],
            'Percentage': [
                self.stats['base_changes'].get(c, 0) / snp_total * 100 if snp_total > 0 else 0
                for c in categories
            ]
        }
        df = pd.DataFrame(data)
        df['Percentage'] = df['Percentage'].round(2)
        return df
    
    def indel_stats_by_bin(self) -> Dict[str, Dict[str, int]]:
        """
        Get indel statistics grouped by size bins.
        
        Returns:
            Dict with 'insertions' and 'deletions' keys, each containing bin -> count mapping
        """
        if not self.stats:
            self.extract_all_stats()
        
        sizes = self.stats['indel_sizes']
        
        # Initialize bins
        bin_stats = {
            'insertions': {bin_label: 0 for bin_label in self.INDEL_SIZE_BINS},
            'deletions': {bin_label: 0 for bin_label in self.INDEL_SIZE_BINS}
        }
        
        # Count indels by bin
        for size in sizes:
            bin_label = self.categorize_indel_size(size)
            if size > 0:
                bin_stats['insertions'][bin_label] += 1
            else:
                bin_stats['deletions'][bin_label] += 1
        
        return bin_stats
    
    def get_indel_size_summary(self) -> pd.DataFrame:
        """Get indel size distribution summary."""
        if not self.stats:
            self.extract_all_stats()
        
        sizes = self.stats['indel_sizes']
        if not sizes:
            return pd.DataFrame({'Metric': [], 'Value': []})
        
        insertions = [s for s in sizes if s > 0]
        deletions = [s for s in sizes if s < 0]
        
        data = {
            'Metric': [
                'Total INDELs', 'Insertions', 'Deletions',
                'Median Size', 'Max Insertion', 'Max Deletion (abs)',
                'Mean Insertion Size', 'Mean Deletion Size (abs)'
            ],
            'Value': [
                len(sizes),
                len(insertions),
                len(deletions),
                np.median(np.abs(sizes)) if sizes else 0,
                max(insertions) if insertions else 0,
                abs(min(deletions)) if deletions else 0,
                np.mean(insertions) if insertions else 0,
                np.mean(np.abs(deletions)) if deletions else 0
            ]
        }
        df = pd.DataFrame(data)
        df['Value'] = df['Value'].round(2)
        return df


print('✓ VCFBenchmarkStats class defined')

## 4. Data Extraction

Parse VCF filenames and extract statistics from all files.

In [ ]:
def parse_benchmark_filename(vcf_path: Path) -> Dict[str, str]:
    """
    Parse benchmarking VCF filename to extract metadata.
    Expected format: Sample_Caller_Model.vcf.gz
    """
    filename = vcf_path.name.replace('.vcf.gz', '').replace('_somatic-only', '')
    
    # Pattern: Sample_Caller_Model (e.g., H1437_DeepSomatic_HCC1395-model)
    pattern = r'^([A-Za-z0-9]+)_([A-Za-z]+)_([A-Za-z0-9-]+)$'
    match = re.match(pattern, filename)
    
    if match:
        sample, caller, model = match.groups()
        return {
            'sample': sample,
            'caller': caller,
            'model': model,
            'filename': vcf_path.name,
            'path': vcf_path
        }
    
    return {
        'sample': filename,
        'caller': 'Unknown',
        'model': 'Unknown',
        'filename': vcf_path.name,
        'path': vcf_path
    }


def extract_all_vcf_stats(vcf_paths: List[Path], verbose: bool = True) -> Dict[str, Dict]:
    """
    Extract statistics from all VCF files.
    
    Returns:
        Dict mapping filename to {'metadata': {...}, 'stats': {...}, 'extractor': VCFBenchmarkStats}
    """
    all_stats = {}
    
    for i, vcf_path in enumerate(vcf_paths):
        if not vcf_path.exists():
            if verbose:
                print(f'⚠️  Skipping missing file: {vcf_path.name}')
            continue
        
        if verbose:
            print(f'[{i+1}/{len(vcf_paths)}] Processing {vcf_path.name}...')
        
        metadata = parse_benchmark_filename(vcf_path)
        extractor = VCFBenchmarkStats(vcf_path)
        stats = extractor.extract_all_stats()
        
        all_stats[vcf_path.name] = {
            'metadata': metadata,
            'stats': stats,
            'extractor': extractor
        }
        
        if verbose:
            print(f'    → {stats["total"]:,} variants (SNP: {stats["snps"]:,}, INDEL: {stats["indels"]:,})')
    
    return all_stats


# Extract statistics from all VCFs
print('Extracting statistics from all VCF files...\n')
all_vcf_stats = extract_all_vcf_stats(valid_vcfs, verbose=True)
print(f'\n✓ Processed {len(all_vcf_stats)} VCF files')

## 5. Whole-Landscape Analysis

Cross-sample statistics and visualizations.

In [ ]:
def create_landscape_summary(all_vcf_stats: Dict) -> pd.DataFrame:
    """
    Create comprehensive summary table with counts and percentages side-by-side.
    """
    rows = []
    
    for filename, data in all_vcf_stats.items():
        metadata = data['metadata']
        stats = data['stats']
        total = stats['total']
        
        # Calculate PASS count and percentage
        pass_count = stats['filters'].get('PASS', 0)
        pass_pct = pass_count / total * 100 if total > 0 else 0
        
        row = {
            'Sample': metadata['sample'],
            'Model': metadata['model'],
            'Total_Variants': total,
            'SNP_Count': stats['snps'],
            'SNP_Pct': stats['snps'] / total * 100 if total > 0 else 0,
            'INDEL_Count': stats['indels'],
            'INDEL_Pct': stats['indels'] / total * 100 if total > 0 else 0,
            'Other_Count': stats['other'],
            'TiTv_Ratio': stats['titv_ratio'],
            'PASS_Count': pass_count,
            'PASS_Pct': pass_pct,
        }
        rows.append(row)
    
    df = pd.DataFrame(rows)
    
    # Round percentages
    pct_cols = [c for c in df.columns if c.endswith('_Pct')]
    df[pct_cols] = df[pct_cols].round(2)
    df['TiTv_Ratio'] = df['TiTv_Ratio'].round(3)
    
    # Sort by Sample then Model
    df = df.sort_values(['Sample', 'Model']).reset_index(drop=True)
    
    return df


# Create landscape summary
landscape_df = create_landscape_summary(all_vcf_stats)

print('=' * 80)
print('LANDSCAPE SUMMARY: All Samples and Models')
print('=' * 80)
display(landscape_df)

# Summary statistics
print(f'\nTotal variants across all files: {landscape_df["Total_Variants"].sum():,}')
print(f'Mean Ti/Tv ratio: {landscape_df["TiTv_Ratio"].mean():.3f}')

### 5.1 Variant Type Distribution

In [ ]:
# Variant type stacked bar chart by sample and model
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Variant Counts by Sample', 'Variant Percentages by Sample'],
    horizontal_spacing=0.1
)

# Prepare data
samples = landscape_df['Sample'].unique()
models = landscape_df['Model'].unique()

colors = {'SNP': '#636EFA', 'INDEL': '#EF553B', 'Other': '#00CC96'}

for model in models:
    model_data = landscape_df[landscape_df['Model'] == model]
    
    # Count plot
    fig.add_trace(
        go.Bar(
            name=f'{model} - SNP',
            x=model_data['Sample'],
            y=model_data['SNP_Count'],
            marker_color=colors['SNP'],
            text=model_data['SNP_Count'].apply(lambda x: f'{x:,}'),
            textposition='inside',
            legendgroup=model,
            showlegend=True
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Bar(
            name=f'{model} - INDEL',
            x=model_data['Sample'],
            y=model_data['INDEL_Count'],
            marker_color=colors['INDEL'],
            text=model_data['INDEL_Count'].apply(lambda x: f'{x:,}'),
            textposition='inside',
            legendgroup=model,
            showlegend=True
        ),
        row=1, col=1
    )
    
    # Percentage plot
    fig.add_trace(
        go.Bar(
            name=f'{model} - SNP %',
            x=model_data['Sample'],
            y=model_data['SNP_Pct'],
            marker_color=colors['SNP'],
            text=model_data['SNP_Pct'].apply(lambda x: f'{x:.1f}%'),
            textposition='inside',
            legendgroup=model + '_pct',
            showlegend=False
        ),
        row=1, col=2
    )
    fig.add_trace(
        go.Bar(
            name=f'{model} - INDEL %',
            x=model_data['Sample'],
            y=model_data['INDEL_Pct'],
            marker_color=colors['INDEL'],
            text=model_data['INDEL_Pct'].apply(lambda x: f'{x:.1f}%'),
            textposition='inside',
            legendgroup=model + '_pct',
            showlegend=False
        ),
        row=1, col=2
    )

fig.update_layout(
    title='Variant Type Distribution by Sample and Model',
    barmode='stack',
    height=500,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5)
)
fig.update_yaxes(title_text='Count', row=1, col=1)
fig.update_yaxes(title_text='Percentage (%)', row=1, col=2)

fig.show()

### 5.2 Ti/Tv Ratio Comparison

In [ ]:
# Ti/Tv ratio comparison bar chart
fig = px.bar(
    landscape_df,
    x='Sample',
    y='TiTv_Ratio',
    color='Model',
    barmode='group',
    title='Ti/Tv Ratio by Sample and Model',
    text='TiTv_Ratio',
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(
    yaxis_title='Ti/Tv Ratio',
    height=450,
    showlegend=True
)

# Add reference lines for expected Ti/Tv ranges
fig.add_hline(y=2.0, line_dash='dash', line_color='gray', 
              annotation_text='Typical WGS (~2.0)', annotation_position='right')
fig.add_hline(y=2.8, line_dash='dash', line_color='lightgray',
              annotation_text='Typical WES (~2.8)', annotation_position='right')

fig.show()

# Ti/Tv summary statistics
print('\nTi/Tv Ratio Summary:')
titv_summary = landscape_df.groupby('Model')['TiTv_Ratio'].agg(['mean', 'std', 'min', 'max']).round(3)
display(titv_summary)

### 5.3 Chromosome Distribution

In [ ]:
def create_chromosome_matrix(all_vcf_stats: Dict) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Create chromosome distribution matrices (counts and percentages).
    Chromosomes are sorted in natural order.
    """
    # Collect all unique chromosomes
    all_chroms = set()
    for data in all_vcf_stats.values():
        all_chroms.update(data['stats']['chromosomes'].keys())
    
    # Natural sort
    sorted_chroms = natural_sort_chromosomes(list(all_chroms))
    
    # Build matrices
    count_data = {}
    pct_data = {}
    
    for filename, data in all_vcf_stats.items():
        sample = data['metadata']['sample']
        model = data['metadata']['model']
        label = f"{sample}_{model.replace('-model', '')}"
        
        total = data['stats']['total']
        chrom_counts = data['stats']['chromosomes']
        
        count_data[label] = {c: chrom_counts.get(c, 0) for c in sorted_chroms}
        pct_data[label] = {c: chrom_counts.get(c, 0) / total * 100 if total > 0 else 0 for c in sorted_chroms}
    
    count_df = pd.DataFrame(count_data).T
    count_df = count_df[sorted_chroms]  # Ensure column order
    
    pct_df = pd.DataFrame(pct_data).T
    pct_df = pct_df[sorted_chroms].round(2)
    
    return count_df, pct_df


# Create chromosome matrices
chrom_count_df, chrom_pct_df = create_chromosome_matrix(all_vcf_stats)

print('Chromosome Distribution (Counts):')
display(chrom_count_df.head())

print('\nChromosome Distribution (Percentages):')
display(chrom_pct_df.head())

In [ ]:
# Chromosome distribution heatmap
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=['Variant Counts by Chromosome', 'Variant Percentage by Chromosome'],
    vertical_spacing=0.15,
    row_heights=[0.5, 0.5]
)

# Count heatmap
fig.add_trace(
    go.Heatmap(
        z=chrom_count_df.values,
        x=chrom_count_df.columns,
        y=chrom_count_df.index,
        colorscale='Blues',
        text=chrom_count_df.values,
        texttemplate='%{text:,}',
        textfont={'size': 8},
        showscale=True,
        colorbar=dict(title='Count', len=0.4, y=0.8)
    ),
    row=1, col=1
)

# Percentage heatmap
fig.add_trace(
    go.Heatmap(
        z=chrom_pct_df.values,
        x=chrom_pct_df.columns,
        y=chrom_pct_df.index,
        colorscale='Greens',
        text=chrom_pct_df.values,
        texttemplate='%{text:.1f}%',
        textfont={'size': 8},
        showscale=True,
        colorbar=dict(title='%', len=0.4, y=0.25)
    ),
    row=2, col=1
)

fig.update_layout(
    title='Chromosome Distribution Across Samples (Natural Sort Order)',
    height=800,
    width=1200
)

fig.show()

### 5.4 FILTER Label Statistics

In [ ]:
def create_filter_summary(all_vcf_stats: Dict) -> pd.DataFrame:
    """Create FILTER label summary across all samples."""
    # Collect all unique filters
    all_filters = set()
    for data in all_vcf_stats.values():
        all_filters.update(data['stats']['filters'].keys())
    
    sorted_filters = sorted(all_filters)
    
    rows = []
    for filename, data in all_vcf_stats.items():
        metadata = data['metadata']
        stats = data['stats']
        total = stats['total']
        
        row = {
            'Sample': metadata['sample'],
            'Model': metadata['model'],
            'Total': total
        }
        
        for filt in sorted_filters:
            count = stats['filters'].get(filt, 0)
            row[f'{filt}_Count'] = count
            row[f'{filt}_Pct'] = count / total * 100 if total > 0 else 0
        
        rows.append(row)
    
    df = pd.DataFrame(rows)
    
    # Round percentages
    pct_cols = [c for c in df.columns if c.endswith('_Pct')]
    df[pct_cols] = df[pct_cols].round(2)
    
    return df.sort_values(['Sample', 'Model']).reset_index(drop=True)


# Create FILTER summary
filter_df = create_filter_summary(all_vcf_stats)

# Display summary
print('FILTER Label Summary:')
display(filter_df)

# Get unique filters for visualization
filter_cols = [c.replace('_Count', '') for c in filter_df.columns if c.endswith('_Count')]

if len(filter_cols) > 0:
    # FILTER distribution grouped bar chart
    fig = go.Figure()
    
    colors = px.colors.qualitative.Set3
    
    for i, filt in enumerate(filter_cols):
        fig.add_trace(go.Bar(
            name=filt,
            x=[f"{row['Sample']}_{row['Model'].replace('-model', '')}" for _, row in filter_df.iterrows()],
            y=filter_df[f'{filt}_Count'],
            marker_color=colors[i % len(colors)],
            text=filter_df[f'{filt}_Pct'].apply(lambda x: f'{x:.1f}%'),
            textposition='auto'
        ))
    
    fig.update_layout(
        title='FILTER Label Distribution (Counts with % Labels)',
        xaxis_title='Sample_Model',
        yaxis_title='Count',
        barmode='group',
        height=500,
        legend=dict(orientation='h', yanchor='bottom', y=1.02)
    )
    fig.show()

### 5.5 Base Change Spectrum (6-Type Mutation Categories)

In [ ]:
def create_base_change_matrix(all_vcf_stats: Dict) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Create base change spectrum matrices (counts and percentages)."""
    categories = ['C>A', 'C>G', 'C>T', 'T>A', 'T>C', 'T>G']
    
    count_data = {}
    pct_data = {}
    
    for filename, data in all_vcf_stats.items():
        sample = data['metadata']['sample']
        model = data['metadata']['model']
        label = f"{sample}_{model.replace('-model', '')}"
        
        base_changes = data['stats']['base_changes']
        snp_total = data['stats']['snps']
        
        count_data[label] = {c: base_changes.get(c, 0) for c in categories}
        pct_data[label] = {c: base_changes.get(c, 0) / snp_total * 100 if snp_total > 0 else 0 for c in categories}
    
    count_df = pd.DataFrame(count_data).T[categories]
    pct_df = pd.DataFrame(pct_data).T[categories].round(2)
    
    return count_df, pct_df


# Create base change matrices
bc_count_df, bc_pct_df = create_base_change_matrix(all_vcf_stats)

# Display tables with counts and percentages side-by-side
print('Base Change Spectrum (Counts):')
display(bc_count_df)

print('\nBase Change Spectrum (Percentages of SNPs):')
display(bc_pct_df)

# Base change spectrum visualization
# Standard mutation spectrum colors
spectrum_colors = {
    'C>A': '#3498db',  # Blue
    'C>G': '#000000',  # Black
    'C>T': '#e74c3c',  # Red
    'T>A': '#95a5a6',  # Gray
    'T>C': '#2ecc71',  # Green
    'T>G': '#f39c12'   # Orange/Pink
}

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Base Change Counts', 'Base Change Percentages'],
    horizontal_spacing=0.1
)

categories = ['C>A', 'C>G', 'C>T', 'T>A', 'T>C', 'T>G']

for cat in categories:
    # Count bars
    fig.add_trace(
        go.Bar(
            name=cat,
            x=bc_count_df.index,
            y=bc_count_df[cat],
            marker_color=spectrum_colors[cat],
            legendgroup=cat,
            showlegend=True
        ),
        row=1, col=1
    )
    
    # Percentage bars
    fig.add_trace(
        go.Bar(
            name=cat,
            x=bc_pct_df.index,
            y=bc_pct_df[cat],
            marker_color=spectrum_colors[cat],
            legendgroup=cat,
            showlegend=False
        ),
        row=1, col=2
    )

fig.update_layout(
    title='6-Type Mutation Spectrum Across Samples',
    barmode='group',
    height=500,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5)
)
fig.update_xaxes(tickangle=45)
fig.update_yaxes(title_text='Count', row=1, col=1)
fig.update_yaxes(title_text='Percentage (%)', row=1, col=2)

fig.show()

### 5.6 Indel Size Distribution

In [ ]:
def create_indel_size_summary(all_vcf_stats: Dict) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Create indel size summary statistics and binned aggregated statistics.
    
    Returns:
        Tuple of (summary_df, binned_df)
    """
    summary_rows = []
    
    # Aggregate binned data across all samples
    aggregated_insertions = {bin_label: 0 for bin_label in VCFBenchmarkStats.INDEL_SIZE_BINS}
    aggregated_deletions = {bin_label: 0 for bin_label in VCFBenchmarkStats.INDEL_SIZE_BINS}
    
    for filename, data in all_vcf_stats.items():
        metadata = data['metadata']
        sizes = data['stats']['indel_sizes']
        
        if not sizes:
            continue
        
        insertions = [s for s in sizes if s > 0]
        deletions = [s for s in sizes if s < 0]
        
        # Get binned stats for this file
        extractor = data['extractor']
        bin_stats = extractor.indel_stats_by_bin()
        
        # Aggregate across all samples
        for bin_label in VCFBenchmarkStats.INDEL_SIZE_BINS:
            aggregated_insertions[bin_label] += bin_stats['insertions'][bin_label]
            aggregated_deletions[bin_label] += bin_stats['deletions'][bin_label]
        
        row = {
            'Sample': metadata['sample'],
            'Model': metadata['model'],
            'Total_INDELs': len(sizes),
            'Insertions_Count': len(insertions),
            'Insertions_Pct': len(insertions) / len(sizes) * 100 if sizes else 0,
            'Deletions_Count': len(deletions),
            'Deletions_Pct': len(deletions) / len(sizes) * 100 if sizes else 0,
            'Median_Size': np.median(np.abs(sizes)),
            'Max_Insertion': max(insertions) if insertions else 0,
            'Max_Deletion': abs(min(deletions)) if deletions else 0,
            'Mean_Insertion': np.mean(insertions) if insertions else 0,
            'Mean_Deletion': np.mean(np.abs(deletions)) if deletions else 0,
        }
        summary_rows.append(row)
    
    summary_df = pd.DataFrame(summary_rows)
    
    # Round values
    for col in ['Insertions_Pct', 'Deletions_Pct', 'Median_Size', 'Mean_Insertion', 'Mean_Deletion']:
        if col in summary_df.columns:
            summary_df[col] = summary_df[col].round(2)
    
    summary_df = summary_df.sort_values(['Sample', 'Model']).reset_index(drop=True)
    
    # Create binned dataframe
    total_insertions = sum(aggregated_insertions.values())
    total_deletions = sum(aggregated_deletions.values())
    
    binned_data = []
    for bin_label in VCFBenchmarkStats.INDEL_SIZE_BINS:
        binned_data.append({
            'Size_Bin': bin_label,
            'Insertions_Count': aggregated_insertions[bin_label],
            'Insertions_Pct': aggregated_insertions[bin_label] / total_insertions * 100 if total_insertions > 0 else 0,
            'Deletions_Count': aggregated_deletions[bin_label],
            'Deletions_Pct': aggregated_deletions[bin_label] / total_deletions * 100 if total_deletions > 0 else 0
        })
    
    binned_df = pd.DataFrame(binned_data)
    binned_df['Insertions_Pct'] = binned_df['Insertions_Pct'].round(2)
    binned_df['Deletions_Pct'] = binned_df['Deletions_Pct'].round(2)
    
    return summary_df, binned_df


# Create indel size summary
indel_summary_df, indel_binned_df = create_indel_size_summary(all_vcf_stats)

print('Indel Size Summary:')
display(indel_summary_df)

print('\nIndel Size Distribution by Bins (Aggregated):')
display(indel_binned_df)

# Indel size distribution: Side-by-side grouped bar chart with counts and percentages
fig = go.Figure()

bins = indel_binned_df['Size_Bin']
insertions_count = indel_binned_df['Insertions_Count']
deletions_count = indel_binned_df['Deletions_Count']
insertions_pct = indel_binned_df['Insertions_Pct']
deletions_pct = indel_binned_df['Deletions_Pct']

# Add insertion bars
fig.add_trace(go.Bar(
    name='Insertions',
    x=bins,
    y=insertions_count,
    marker_color='#00CC96',
    text=[f'{count:,}<br>({pct:.1f}%)' for count, pct in zip(insertions_count, insertions_pct)],
    textposition='outside',
    offsetgroup=0
))

# Add deletion bars
fig.add_trace(go.Bar(
    name='Deletions',
    x=bins,
    y=deletions_count,
    marker_color='#EF553B',
    text=[f'{count:,}<br>({pct:.1f}%)' for count, pct in zip(deletions_count, deletions_pct)],
    textposition='outside',
    offsetgroup=1
))

fig.update_layout(
    title='Indel Size Distribution by Bins (Aggregated Across All Samples)',
    xaxis=dict(title='Size Bin (bp)', tickmode='linear'),
    yaxis=dict(title='Count'),
    barmode='group',
    height=600,
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='center',
        x=0.5
    ),
    hovermode='x unified'
)

fig.show()

print('\n📊 Indel Statistics Summary:')
print(f'Total Insertions: {indel_binned_df["Insertions_Count"].sum():,}')
print(f'Total Deletions: {indel_binned_df["Deletions_Count"].sum():,}')
print(f'Most common insertion bin: {indel_binned_df.loc[indel_binned_df["Insertions_Count"].idxmax(), "Size_Bin"]}')
print(f'Most common deletion bin: {indel_binned_df.loc[indel_binned_df["Deletions_Count"].idxmax(), "Size_Bin"]}')

### 5.7 Model Comparison Summary

In [ ]:
# Model comparison: aggregate statistics by model
model_comparison = landscape_df.groupby('Model').agg({
    'Total_Variants': ['sum', 'mean', 'std'],
    'SNP_Count': 'sum',
    'SNP_Pct': 'mean',
    'INDEL_Count': 'sum',
    'INDEL_Pct': 'mean',
    'TiTv_Ratio': ['mean', 'std'],
    'PASS_Pct': 'mean'
}).round(2)

model_comparison.columns = ['_'.join(col).strip() for col in model_comparison.columns.values]

print('=' * 80)
print('MODEL COMPARISON: Aggregate Statistics')
print('=' * 80)
display(model_comparison)

# Model comparison visualization
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=['Total Variants', 'Mean Ti/Tv Ratio', 'Mean SNP Percentage'],
    horizontal_spacing=0.1
)

models = model_comparison.index.tolist()
colors = px.colors.qualitative.Set2[:len(models)]

# Total variants
fig.add_trace(
    go.Bar(
        x=models,
        y=model_comparison['Total_Variants_sum'],
        marker_color=colors,
        text=model_comparison['Total_Variants_sum'].apply(lambda x: f'{x:,.0f}'),
        textposition='outside'
    ),
    row=1, col=1
)

# Mean Ti/Tv
fig.add_trace(
    go.Bar(
        x=models,
        y=model_comparison['TiTv_Ratio_mean'],
        marker_color=colors,
        text=model_comparison['TiTv_Ratio_mean'].apply(lambda x: f'{x:.2f}'),
        textposition='outside',
        error_y=dict(type='data', array=model_comparison['TiTv_Ratio_std'].values)
    ),
    row=1, col=2
)

# Mean SNP percentage
fig.add_trace(
    go.Bar(
        x=models,
        y=model_comparison['SNP_Pct_mean'],
        marker_color=colors,
        text=model_comparison['SNP_Pct_mean'].apply(lambda x: f'{x:.1f}%'),
        textposition='outside'
    ),
    row=1, col=3
)

fig.update_layout(
    title='Model Comparison Overview',
    height=400,
    showlegend=False
)
fig.update_yaxes(title_text='Count', row=1, col=1)
fig.update_yaxes(title_text='Ti/Tv Ratio', row=1, col=2)
fig.update_yaxes(title_text='Percentage (%)', row=1, col=3)

fig.show()

## 6. Per-Sample Analysis

Detailed statistics for each individual sample.

In [ ]:
def create_sample_report(sample_name: str, all_vcf_stats: Dict) -> Dict[str, Any]:
    """
    Create comprehensive report for a single sample across all models.
    """
    # Get all entries for this sample
    sample_data = {
        fname: data for fname, data in all_vcf_stats.items()
        if data['metadata']['sample'] == sample_name
    }
    
    if not sample_data:
        return None
    
    report = {
        'sample': sample_name,
        'models': {},
        'combined': {}
    }
    
    # Per-model statistics
    all_chroms = defaultdict(int)
    all_base_changes = defaultdict(int)
    all_indel_sizes = []
    total_snps = 0
    total_indels = 0
    total_variants = 0
    
    for fname, data in sample_data.items():
        model = data['metadata']['model']
        stats = data['stats']
        
        report['models'][model] = {
            'total': stats['total'],
            'snps': stats['snps'],
            'indels': stats['indels'],
            'titv_ratio': stats['titv_ratio'],
            'filters': stats['filters'],
            'chromosomes': stats['chromosomes'],
            'base_changes': stats['base_changes'],
            'indel_sizes': stats['indel_sizes']
        }
        
        # Aggregate
        total_variants += stats['total']
        total_snps += stats['snps']
        total_indels += stats['indels']
        
        for c, count in stats['chromosomes'].items():
            all_chroms[c] += count
        for bc, count in stats['base_changes'].items():
            all_base_changes[bc] += count
        all_indel_sizes.extend(stats['indel_sizes'])
    
    report['combined'] = {
        'total_variants': total_variants,
        'total_snps': total_snps,
        'total_indels': total_indels,
        'chromosomes': dict(all_chroms),
        'base_changes': dict(all_base_changes),
        'indel_sizes': all_indel_sizes
    }
    
    return report


def display_sample_analysis(sample_name: str, all_vcf_stats: Dict):
    """Display comprehensive analysis for a single sample."""
    report = create_sample_report(sample_name, all_vcf_stats)
    
    if not report:
        print(f'No data found for sample: {sample_name}')
        return
    
    print('=' * 80)
    print(f'SAMPLE: {sample_name}')
    print('=' * 80)
    
    # Summary table
    summary_rows = []
    for model, stats in report['models'].items():
        total = stats['total']
        summary_rows.append({
            'Model': model,
            'Total': total,
            'SNP_Count': stats['snps'],
            'SNP_Pct': f"{stats['snps']/total*100:.1f}%" if total > 0 else '0%',
            'INDEL_Count': stats['indels'],
            'INDEL_Pct': f"{stats['indels']/total*100:.1f}%" if total > 0 else '0%',
            'Ti/Tv': f"{stats['titv_ratio']:.3f}" if stats['titv_ratio'] else 'N/A'
        })
    
    summary_df = pd.DataFrame(summary_rows)
    print('\n📊 Variant Type Summary:')
    display(summary_df)
    
    # Create visualizations
    fig = make_subplots(
        rows=2, cols=2,
        specs=[[{'type': 'pie'}, {'type': 'bar'}],
               [{'type': 'bar'}, {'type': 'histogram'}]],
        subplot_titles=[
            'Variant Type Distribution (Combined)',
            'Chromosome Distribution (Top 10)',
            'Base Change Spectrum',
            'Indel Size Distribution'
        ]
    )
    
    combined = report['combined']
    
    # 1. Variant type pie chart
    fig.add_trace(
        go.Pie(
            labels=['SNP', 'INDEL'],
            values=[combined['total_snps'], combined['total_indels']],
            marker_colors=['#636EFA', '#EF553B'],
            textinfo='label+percent+value',
            hole=0.3
        ),
        row=1, col=1
    )
    
    # 2. Chromosome bar chart (top 10, natural sorted)
    chroms = natural_sort_chromosomes(list(combined['chromosomes'].keys()))
    chrom_counts = [combined['chromosomes'][c] for c in chroms]
    total_chrom = sum(chrom_counts)
    
    # Take top 10 chromosomes by count
    chrom_data = sorted(zip(chroms, chrom_counts), key=lambda x: x[1], reverse=True)[:10]
    top_chroms, top_counts = zip(*chrom_data) if chrom_data else ([], [])
    
    fig.add_trace(
        go.Bar(
            x=list(top_chroms),
            y=list(top_counts),
            marker_color='#636EFA',
            text=[f'{c:,}<br>({c/total_chrom*100:.1f}%)' for c in top_counts],
            textposition='outside'
        ),
        row=1, col=2
    )
    
    # 3. Base change spectrum
    bc_categories = ['C>A', 'C>G', 'C>T', 'T>A', 'T>C', 'T>G']
    bc_colors = ['#3498db', '#000000', '#e74c3c', '#95a5a6', '#2ecc71', '#f39c12']
    bc_counts = [combined['base_changes'].get(c, 0) for c in bc_categories]
    snp_total = combined['total_snps']
    
    fig.add_trace(
        go.Bar(
            x=bc_categories,
            y=bc_counts,
            marker_color=bc_colors,
            text=[f'{c:,}<br>({c/snp_total*100:.1f}%)' if snp_total > 0 else f'{c:,}' for c in bc_counts],
            textposition='outside'
        ),
        row=2, col=1
    )
    
    # 4. Indel size histogram
    if combined['indel_sizes']:
        fig.add_trace(
            go.Histogram(
                x=combined['indel_sizes'],
                nbinsx=50,
                marker_color='#00CC96'
            ),
            row=2, col=2
        )
    
    fig.update_layout(
        title=f'Sample Analysis: {sample_name}',
        height=800,
        showlegend=False
    )
    
    fig.show()
    
    return report


# Get unique samples
unique_samples = list(set(data['metadata']['sample'] for data in all_vcf_stats.values()))
print(f'Found {len(unique_samples)} unique samples: {sorted(unique_samples)}')

In [ ]:
# Generate per-sample analysis for each sample
sample_reports = {}

for sample in sorted(unique_samples):
    report = display_sample_analysis(sample, all_vcf_stats)
    if report:
        sample_reports[sample] = report
    print('\n')

## 7. Combined Summary and Export

Export all statistics to Excel and CSV formats.

In [ ]:
def export_all_statistics(
    landscape_df: pd.DataFrame,
    chrom_count_df: pd.DataFrame,
    chrom_pct_df: pd.DataFrame,
    bc_count_df: pd.DataFrame,
    bc_pct_df: pd.DataFrame,
    filter_df: pd.DataFrame,
    indel_summary_df: pd.DataFrame,
    indel_binned_df: pd.DataFrame,
    output_dir: Path
):
    """Export all statistics to multi-sheet Excel file."""
    
    excel_path = output_dir / 'vcf_benchmarking_statistics.xlsx'
    
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        # Landscape summary
        landscape_df.to_excel(writer, sheet_name='Landscape_Summary', index=False)
        
        # Chromosome distributions
        chrom_count_df.to_excel(writer, sheet_name='Chromosome_Counts')
        chrom_pct_df.to_excel(writer, sheet_name='Chromosome_Percentages')
        
        # Base change spectrum
        bc_count_df.to_excel(writer, sheet_name='BaseChange_Counts')
        bc_pct_df.to_excel(writer, sheet_name='BaseChange_Percentages')
        
        # FILTER labels
        filter_df.to_excel(writer, sheet_name='Filter_Labels', index=False)
        
        # Indel sizes
        indel_summary_df.to_excel(writer, sheet_name='Indel_Summary', index=False)
        indel_binned_df.to_excel(writer, sheet_name='Indel_Binned', index=False)
    
    print(f'✓ Excel exported: {excel_path}')
    
    # Also export individual CSVs
    csv_dir = output_dir / 'csv'
    csv_dir.mkdir(exist_ok=True)
    
    landscape_df.to_csv(csv_dir / 'landscape_summary.csv', index=False)
    chrom_count_df.to_csv(csv_dir / 'chromosome_counts.csv')
    chrom_pct_df.to_csv(csv_dir / 'chromosome_percentages.csv')
    bc_count_df.to_csv(csv_dir / 'basechange_counts.csv')
    bc_pct_df.to_csv(csv_dir / 'basechange_percentages.csv')
    filter_df.to_csv(csv_dir / 'filter_labels.csv', index=False)
    indel_summary_df.to_csv(csv_dir / 'indel_summary.csv', index=False)
    indel_binned_df.to_csv(csv_dir / 'indel_binned.csv', index=False)
    
    print(f'✓ CSV files exported to: {csv_dir}')
    
    return excel_path


# Export all statistics
excel_path = export_all_statistics(
    landscape_df=landscape_df,
    chrom_count_df=chrom_count_df,
    chrom_pct_df=chrom_pct_df,
    bc_count_df=bc_count_df,
    bc_pct_df=bc_pct_df,
    filter_df=filter_df,
    indel_summary_df=indel_summary_df,
    indel_binned_df=indel_binned_df,
    output_dir=OUTPUT_DIR
)

### 7.1 Final Summary Dashboard

In [ ]:
# Create final summary dashboard
print('=' * 80)
print('FINAL SUMMARY DASHBOARD')
print('=' * 80)

# Overall statistics
total_variants = landscape_df['Total_Variants'].sum()
total_snps = landscape_df['SNP_Count'].sum()
total_indels = landscape_df['INDEL_Count'].sum()
mean_titv = landscape_df['TiTv_Ratio'].mean()

print(f'''
📊 OVERALL STATISTICS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Total VCF Files Analyzed:  {len(all_vcf_stats)}
Total Unique Samples:      {len(unique_samples)}
Total Models:              {len(landscape_df['Model'].unique())}

Total Variants:            {total_variants:,}
  • SNPs:                  {total_snps:,} ({total_snps/total_variants*100:.1f}%)
  • INDELs:                {total_indels:,} ({total_indels/total_variants*100:.1f}%)

Mean Ti/Tv Ratio:          {mean_titv:.3f}
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
''')

# Cross-tabulation summary
print('\n📋 CROSS-TABULATION: Sample × Key Metrics')
cross_tab = landscape_df.pivot_table(
    index='Sample',
    columns='Model',
    values=['Total_Variants', 'SNP_Pct', 'TiTv_Ratio'],
    aggfunc='first'
)
display(cross_tab.round(2))

# Top chromosomes across all samples
print('\n🧬 TOP 5 CHROMOSOMES BY VARIANT COUNT (All Samples Combined):')
combined_chroms = chrom_count_df.sum().sort_values(ascending=False).head(5)
for chrom, count in combined_chroms.items():
    pct = count / total_variants * 100
    print(f'  {chrom}: {count:,} ({pct:.1f}%)')

# Base change spectrum summary
print('\n🔬 BASE CHANGE SPECTRUM SUMMARY (All Samples Combined):')
combined_bc = bc_count_df.sum().sort_values(ascending=False)
for bc, count in combined_bc.items():
    pct = count / total_snps * 100 if total_snps > 0 else 0
    print(f'  {bc}: {count:,} ({pct:.1f}%)')

print(f'''
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✓ Analysis complete!
✓ Results exported to: {OUTPUT_DIR}
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
''')